In [ ]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 36.5 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 636 kB 47.4 MB/s 


In [ ]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


TRAIN D

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_train_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/train/trainD.tsv", header=None, sep="\t")

df_train_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/train/text_files') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_train_clinical_P)):
    df_2 = load_articles(df_train_clinical_P, i)
    df_train_P = df_train_P.append(df_2)
    count = count + 1


df_train_P = df_train_P.set_axis(np.array(range(0,len(df_train_clinical_P))))
df_train_P[1] = df_train_clinical_P[1]
df_train_P.columns = ['content', 'labels']
df_train_P.head(10)

train = df_train_P

DEV D

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_dev_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/dev/devD.tsv", header=None, sep="\t")

df_dev_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/dev/text_files') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_dev_clinical_P)):
    df_2 = load_articles(df_dev_clinical_P, i)
    df_dev_P = df_dev_P.append(df_2)
    count = count + 1


df_dev_P = df_dev_P.set_axis(np.array(range(0,len(df_dev_clinical_P))))
df_dev_P[1] = df_dev_clinical_P[1]
df_dev_P.columns = ['content', 'labels']
df_dev_P.head(10)

dev = df_dev_P

TEST D

In [ ]:
from pathlib import Path
df_test_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/test/testD.tsv", header=None, sep="\t")

df_test_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/test/text_files') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n", dtype='unicode', error_bad_lines=False)
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_test_clinical_P)):
    df_2 = load_articles(df_test_clinical_P, i)
    df_test_P = df_test_P.append(df_2)
    count = count + 1


df_test_P = df_test_P.set_axis(np.array(range(0,len(df_test_clinical_P))))
df_test_P[1] = df_test_clinical_P[1]
LABEL_COLUMNS = ['content', 'labels']
df_test_P.columns = LABEL_COLUMNS
df_test_P.head(10)

test = df_test_P

**MATRIX TRANSFORMATION**

In [ ]:
new_labels_train = []
new_labels_test = []
new_labels_dev = []
lista = ['content']
for i in range(len(df_train_P['labels'])):
    new_labels_train.append([df_train_P['labels'][i]])
    lista = lista + new_labels_train[i]
for i in range(len(df_test_P['labels'])):
    new_labels_test.append([df_test_P['labels'][i]])
    lista = lista + new_labels_test[i]
for i in range(len(df_dev_P['labels'])):
    new_labels_dev.append([df_dev_P['labels'][i]])
    lista = lista + new_labels_dev[i]

    
####### TRAIN #######
myvec = np.zeros((len(df_train_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf = pd.DataFrame(data = myvec, index=range(len(df_train_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf.columns = lista
auxdf = auxdf.loc[:,~auxdf.columns.duplicated()]

df_train_P1 = df_train_P
df_train_P = df_train_P.drop_duplicates('content', ignore_index = True)
auxdf['content'] = df_train_P['content']

for j in range(len(auxdf['content'])):
    for i in range(len(df_train_P1['content'])):
        label = df_train_P1['labels'][i]
        if auxdf['content'][j]==df_train_P1['content'][i]:
            label = df_train_P1['labels'][i]
            auxdf[label][j] = 1 
            
train_matrix = auxdf            


#### TEST #####
myvec1 = np.zeros((len(df_test_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf1 = pd.DataFrame(data = myvec1, index=range(len(df_test_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf1.columns = lista
auxdf1 = auxdf1.loc[:,~auxdf1.columns.duplicated()]

df_test_P1 = df_test_P
df_test_P = df_test_P.drop_duplicates('content', ignore_index = True)
auxdf1['content'] = df_test_P['content']

for j in range(len(auxdf1['content'])):
    for i in range(len(df_test_P1['content'])):
        label1 = df_test_P1['labels'][i]
        if auxdf1['content'][j]==df_test_P1['content'][i]:
            label1 = df_test_P1['labels'][i]
            auxdf1[label1][j] = 1 
            
test_matrix = auxdf1            


###### DEV ######
myvec3 = np.zeros((len(df_dev_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf3 = pd.DataFrame(data = myvec3, index=range(len(df_dev_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf3.columns = lista
auxdf3 = auxdf3.loc[:,~auxdf3.columns.duplicated()]

df_dev_P1 = df_dev_P
df_dev_P = df_dev_P.drop_duplicates('content', ignore_index = True)
auxdf3['content'] = df_dev_P['content']

for j in range(len(auxdf3['content'])):
    for i in range(len(df_dev_P1['content'])):
        label3 = df_dev_P1['labels'][i]
        if auxdf3['content'][j]==df_dev_P1['content'][i]:
            label = df_dev_P1['labels'][i]
            auxdf3[label3][j] = 1 
            
dev_matrix = auxdf3            


##### TRAIN MATRIX #####
df_train_clinical = pd.DataFrame()
# train_matrix = pd.concat([train_matrix, dev_matrix], ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                               content  n44.8  ...  d13.5  n81.2
0    Describimos el caso de un varón de 37 años con...    1.0  ...    0.0    0.0
1    Se trata de una mujer de 29 años sometida a un...    0.0  ...    0.0    0.0
2    Varón de 36 años, sin antecedentes de interés,...    0.0  ...    0.0    0.0
3    Mujer de 42 años en el momento de someterse a ...    0.0  ...    0.0    0.0
4    Varón de 65 años que consultaba por un síndrom...    0.0  ...    0.0    0.0
..                                                 ...    ...  ...    ...    ...
495  Describimos el caso de una paciente de 47 años...    0.0  ...    0.0    0.0
496  Paciente varón de 65 años diagnosticado de OI ...    0.0  ...    0.0    0.0
497  Varón de 72 años en programa de HD desde dicie...    0.0  ...    0.0    0.0
498  Mujer de 33 años, que llega a urgencias con un...    0.0  ...    0.0    0.0
499  Presentamos el caso de una paciente de 62 años...    0.0  ...    0.0    0.0

[500 rows x 2558 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                               content  n44.8  ...  d13.5  n81.2
0    Paciente de 70 años de edad, minero jubilado, ...    0.0  ...    0.0    0.0
1    Varón de diecisiete años sin antecedentes de i...    0.0  ...    0.0    0.0
2    Paciente femenina de 62 años de edad, obesa, d...    0.0  ...    0.0    0.0
3    Se trata de un varón de 53 años sin antecedent...    0.0  ...    0.0    0.0
4    Paciente varón, de 40 años de edad, con antece...    0.0  ...    0.0    0.0
..                                                 ...    ...  ...    ...    ...
245  El caso que describe nuestro trabajo es el de ...    0.0  ...    0.0    0.0
246  Varón de 22 años (68 kg., 172 cm.), militar en...    0.0  ...    0.0    0.0
247  Varón de 53 años de edad, ciclista veterano de...    0.0  ...    0.0    0.0
248  Varón de 34 años que consultó por dolor con pa...    0.0  ...    0.0    0.0
249  Varón de 57 años con antecedentes de hipertens...    0.0  ...    0.0    0.0

[250 rows x 2558 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                               content  n44.8  ...  d13.5  n81.2
0    Mujer de 29 años con antecedentes de ulcus duo...    0.0  ...    0.0    0.0
1    Varón de 58 años de edad en el momento del tra...    0.0  ...    0.0    0.0
2    Paciente varón de 22 años de edad, sin anteced...    0.0  ...    0.0    0.0
3    Paciente de 35 años que nos fue remitido al se...    0.0  ...    0.0    0.0
4    Paciente de 90 años que acude a su Urólogo de ...    0.0  ...    0.0    0.0
..                                                 ...    ...  ...    ...    ...
245  Paciente varón de cinco años de edad que tras ...    0.0  ...    0.0    0.0
246  Se trata de un adolescente de 16 años, jugador...    0.0  ...    0.0    0.0
247  Describimos el caso de una paciente de 58 años...    0.0  ...    0.0    0.0
248  Presentamos el caso de un varón de 64 años con...    0.0  ...    0.0    0.0
249  Mujer de 73 años de edad con antecedentes pers...    0.0  ...    1.0    1.0

[250 rows x 2558 columns]
 

In [ ]:
train_matrix1 = train_matrix
test_matrix1 = test_matrix
dev_matrix1 = dev_matrix

In [ ]:
total = []
for i in range(train_matrix1.shape[1]):
    if i>0:
        total.append(sum(train_matrix1.iloc[:,i]))


total_matrix = pd.concat([train_matrix1, test_matrix1, dev_matrix1], ignore_index=True)
lista = total_matrix.columns
lista = lista.delete(0)

mat = pd.DataFrame(total, index = lista)
mat = mat.sort_values(0, ascending = False)

maxi = len(lista)
#maxi = 161

coger = mat.iloc[:maxi]
coger = coger.index
coger = coger.insert(0,'content')

train_matrix = train_matrix[coger]
test_matrix = test_matrix[coger]
dev_matrix = dev_matrix[coger]
# print(train_matrix)
# print(test_matrix)

In [ ]:
!pip install texthero
!pip install tweet-preprocessor

     |████████████████████████████████| 241 kB 6.9 MB/s 
     |████████████████████████████████| 1.5 MB 38.8 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import re 
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob,Word
from nltk.corpus import words
nltk.download('words')
import texthero as hero
import re
from texthero import stopwords

from nltk.corpus import wordnet

import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import tensorflow as tf


def lemma_per_pos(sent):
    '''function to lemmatize according to part of speech tag'''
    tweet_tokenizer=TweetTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = [lemmatizer.lemmatize(w) for w in  tweet_tokenizer.tokenize(sent)]
    return " ".join(lemmatized_list)

def df_preprocessing(df,feature_col):
    '''
    Preprocessing of dataframe
    '''
    stop = set(stopwords.words('spanish'))
    df[feature_col]= (df[feature_col].pipe(hero.lowercase).
                      pipe(hero.remove_urls).
                      pipe(hero.remove_digits).
                      pipe(hero.remove_punctuation).
                      pipe(hero.remove_html_tags) )
    # lemmatization
#     df[feature_col]= [lemma_per_pos(sent) for sent in df[feature_col]]
    # df[col_name]= hero.remove_stopwords(df[col_name],custom_stopwords)
    return df

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
target_col= train_matrix.columns[1:]
feature_col= train_matrix.columns[0]

In [ ]:
with tf.device('/GPU:0'):
    proc_train_df= df_preprocessing(train_matrix,feature_col)

In [ ]:
proc_test_df = df_preprocessing(test_matrix,feature_col)

In [ ]:
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import AdamW, get_linear_schedule_with_warmup
#import tensorflow_addons as tfa


#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-multilingual-uncased'):
  '''Function to create the tokenizer'''

  tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
  return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_col,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_col].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)


    return [input_ids,attention_mask]

def bert_model(pretrained_weights,max_len,learning_rate):
  '''BERT model creation with pretrained weights
  INPUT:
  pretrained_weights: Language model pretrained weights
  max_len: input length '''
  print('Model selected:', pretrained_weights)
  bert=TFAutoModel.from_pretrained(pretrained_weights)
  
  # This is must if you would like to train the layers of language models too.
  for layer in bert.layers:
      layer.trainable = True

  # parameter declaration
#   step = tf.Variable(0, trainable=False)
#   schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [2e-0, 2e-1, 1e-2])
#   # lr and wd can be a function or a tensor
#   lr = learning_rate * schedule(step)
#   wd = lambda:lr * schedule(step)
#   optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

  optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
#   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # declaring inputs, BERT take input_ids and attention_mask as input
  input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
  attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

  bert= bert(input_ids,attention_mask=attention_mask)
  x= bert[0][:,0,:]
  x=tf.keras.layers.Dropout(0.1)(x)
  x= tf.keras.layers.Dense(128)(x)
  x=tf.keras.layers.Dense(64)(x)
  x=tf.keras.layers.Dense(32)(x)

  output=tf.keras.layers.Dense(maxi,activation='sigmoid')(x)

  model=Model(inputs=[input_ids,attention_mask],outputs=[output])
  # compiling model 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE,name='binary_crossentropy'),
                metrics=['accuracy'])
  return model


In [ ]:
pretrained_weights='bert-base-multilingual-uncased'
max_len=256
epochs=20
learning_rate=2e-5
batch_size=4

In [ ]:
tokenizer= create_tokenizer(pretrained_weights)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
x_train= data_tokenization(proc_train_df,feature_col,max_len,tokenizer)

In [ ]:
y_train= proc_train_df[target_col].values
y_train

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
bert=bert_model(pretrained_weights,max_len,learning_rate)
bert.summary()

Model selected: bert-base-multilingual-uncased


Downloading:   0%|          | 0.00/999M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"



125/125 [==============================] - 72s 485ms/step - loss: 0.6209 - accuracy: 0.0000e+00
Epoch 2/20
125/125 [==============================] - 61s 491ms/step - loss: 0.3332 - accuracy: 0.0000e+00
Epoch 3/20
125/125 [==============================] - 61s 488ms/step - loss: 0.1135 - accuracy: 0.0000e+00
Epoch 4/20
125/125 [==============================] - 61s 489ms/step - loss: 0.0457 - accuracy: 0.0000e+00
Epoch 5/20
125/125 [==============================] - 61s 488ms/step - loss: 0.0319 - accuracy: 0.0000e+00
Epoch 6/20
125/125 [==============================] - 61s 489ms/step - loss: 0.0285 - accuracy: 0.0920
Epoch 7/20
125/125 [==============================] - 61s 486ms/step - loss: 0.0270 - accuracy: 0.1600
Epoch 8/20
125/125 [==============================] - 61s 489ms/step - loss: 0.0261 - accuracy: 0.1720
Epoch 9/20
125/125 [==============================] - 61s 490ms/step - loss: 0.0255 - accuracy: 0.2140
Epoch 10/20
125/125 [==============================] - 61s 488ms

In [ ]:
x_test= data_tokenization(test_matrix,feature_col,max_len,tokenizer)
x_test

[array([[  101, 72496, 10102, ...,   167, 19702,   102],
        [  101, 10334, 10273, ..., 43424, 14535,   102],
        [  101, 72496, 46518, ..., 10102,   166,   102],
        ...,
        [  101, 10334, 10273, ..., 66266, 40003,   102],
        [  101, 10334, 10273, ..., 10107, 10102,   102],
        [  101, 10334, 10273, ..., 14181, 10252,   102]], dtype=int32),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int32)]

In [ ]:
preds= bert.predict(x_test)
submiss_df= pd.DataFrame(preds, columns= target_col)
submiss_df

,r52,r69,r50.9,i10,r60.9,r59.9,r53.1,r11.10,r59.0,r10.9,b99.9,r58,d64.9,e11.9,i96,r63.4,r31.9,l53.9,b20,n28.9,r23.1,d72.829,f17.210,t14.90,r51,b19.20,t14.8,r06.00,k65.9,n28.89,l98.9,k75.9,r63.0,n18.9,l90.5,i82.90,r19.7,e80.7,b19.10,r60.0,...,b95.6,l02.32,d12.3,b55.0,s42.002,s02.40f,v23.9xx,s02.81x,s22.20x,s22.5xx,s02.2xx,s32.591,e80.4,c77.1,c18.0,d13.6,k86.2,d16.21,c30.0,s63.006,s62.12,q83.9,o92.6,p07.39,p84,t68,p83.8,z79.810,j39.2,s81.001,z80.1,j34.9,c83.00,c22.9,m33.90,c25.9,m79.5,w01.0xx,m70.51,n81.2
0,0.211322,0.200010,0.217886,0.151189,0.099512,0.193963,0.099118,0.065198,0.167649,0.080283,0.129425,0.080748,0.098549,0.132480,0.072904,0.081541,0.059377,0.082202,0.050798,0.051632,0.060702,0.059168,0.112846,0.063474,0.036552,0.060272,0.033953,0.047940,0.058530,0.057179,0.051858,0.027869,0.063488,0.068119,0.036034,0.040211,0.029071,0.037906,0.032749,0.021674,...,0.000161,0.000091,0.000736,0.000586,0.001056,0.000142,0.000840,0.000632,0.000076,0.000038,0.000184,0.000173,0.000016,0.001330,0.000572,0.000164,0.001026,0.000016,0.002634,0.000953,0.000232,0.000614,0.000703,0.000015,0.000164,0.000526,0.000297,0.000454,0.000487,0.000396,0.000231,0.000610,0.001020,0.001540,0.000120,0.000482,0.000381,0.000638,0.000026,0.000904
1,0.192812,0.188107,0.155405,0.130861,0.068030,0.138637,0.061225,0.060037,0.114997,0.089007,0.079930,0.076873,0.080854,0.079129,0.034797,0.050485,0.070091,0.062820,0.039667,0.050181,0.030110,0.043582,0.066484,0.045793,0.027985,0.040662,0.023398,0.029272,0.040608,0.044243,0.034153,0.022188,0.050268,0.029110,0.029119,0.022762,0.025616,0.027507,0.021354,0.020848,...,0.000078,0.000022,0.000347,0.000210,0.000472,0.000051,0.000325,0.000287,0.000019,0.000009,0.000079,0.000068,0.000004,0.000746,0.000232,0.000057,0.000490,0.000004,0.001163,0.000401,0.000083,0.000218,0.000229,0.000003,0.000051,0.000200,0.000092,0.000200,0.000203,0.000197,0.000065,0.000262,0.000467,0.001043,0.000046,0.000205,0.000230,0.000282,0.000007,0.000445
2,0.221109,0.195499,0.128585,0.277787,0.078936,0.153642,0.086233,0.069384,0.102548,0.093949,0.069144,0.066637,0.092030,0.091326,0.069850,0.069702,0.055918,0.067472,0.046243,0.056768,0.040802,0.048241,0.065012,0.046004,0.042749,0.045423,0.035397,0.032550,0.059758,0.053241,0.049918,0.024230,0.041403,0.038413,0.027825,0.028954,0.023669,0.032844,0.023909,0.026025,...,0.000126,0.000064,0.000588,0.000311,0.000542,0.000070,0.000336,0.000539,0.000042,0.000018,0.000156,0.000112,0.000010,0.000966,0.000328,0.000109,0.000540,0.000008,0.001798,0.000658,0.000148,0.000364,0.000476,0.000005,0.000084,0.000389,0.000229,0.000246,0.000344,0.000346,0.000145,0.000398,0.001021,0.001333,0.000098,0.000359,0.000317,0.000561,0.000018,0.000658
3,0.237196,0.201040,0.176908,0.189525,0.092445,0.236438,0.062469,0.055617,0.181655,0.106359,0.118544,0.091581,0.100190,0.103355,0.065136,0.089098,0.061815,0.090390,0.052365,0.050830,0.029763,0.056985,0.122097,0.061406,0.039825,0.066244,0.022810,0.045794,0.078982,0.070477,0.044262,0.030040,0.058287,0.055005,0.031073,0.044668,0.030005,0.044813,0.038680,0.036612,...,0.000299,0.000082,0.000763,0.000804,0.001050,0.000154,0.000992,0.000797,0.000094,0.000030,0.000226,0.000164,0.000012,0.001186,0.000575,0.000105,0.000779,0.000017,0.002921,0.000987,0.000201,0.000515,0.000724,0.000012,0.000148,0.000550,0.000312,0.000630,0.000556,0.000395,0.000193,0.000574,0.001641,0.002676,0.000101,0.000626,0.000367,0.000807,0.000032,0.000997
4,0.190421,0.151923,0.173161,0.095149,0.091346,0.102483,0.071666,0.064060,0.085262,0.054159,0.062567,0.063082,0.058120,0.065181,0.042494,0.040853,0.051517,0.052885,0.031707,0.035448,0.032882,0.035835,0.044725,0.039562,0.027617,0.028074,0.029488,0.027110,0.028617,0.030325,0.028743,0.020491,0.034216,0.022752,0.022428,0.018853,0.022134,0.025511,0.020168,0.017826,...,0.000039,0.000011,0.000172,0.000126,0.000238,0.000031,0.000219,0.000157,0.000009,0.000006,0.000050,0.000032,0.000002,0.000442,0.000141,0.000030,0.000282,0.000002,0.000979,0.000240,0.000050,0.000123,0.000165,0.000001,0.000

In [ ]:
outputs = submiss_df>0.5
f1_score_micro = metrics.f1_score(test_matrix.iloc[:,1:], outputs, average='micro')
f1_score_macro = metrics.f1_score(test_matrix.iloc[:,1:], outputs, average='macro')
ap = metrics.average_precision_score(test_matrix.iloc[:,1:], outputs, average='micro')
print(f1_score_micro)
print(f1_score_macro)
print(ap)

0.0
0.0
0.004445834962847087


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.

